### 共线性对模型有如下4种效应

- 参数的估计值变得不准确
- 参数估计值的标准差变大
- 参数显著性检验变得不准确，容易将重要的自变量误判为不显著，即针对模型参数的假设检验变得不准确
- 对于已知数据，模型的预测效果几乎不受影响

In [1]:
import numpy as np
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.decomposition import PCA
from sklearn import datasets


dataset = datasets.make_classification(n_samples=5000, n_features=15, n_informative=5, n_redundant=2, n_repeated=2, 
                                       n_classes=2, n_clusters_per_class=2, shuffle=True, random_state=2018)
X, y = dataset[0], dataset[1]

In [3]:
import scipy.stats.stats as scss

# 第一个值是相关系数，第二个值是P值
scss.pearsonr(X[:, 0], X[:, 1])

(-0.5593271141268006, 0.0)

In [4]:
scss.spearmanr(X[:, 0], X[:, 1])

SpearmanrResult(correlation=-0.48175831850233264, pvalue=5.732959707734371e-289)

### 1 岭回归(L2正则项)

- 在线性回归模型中加入L2惩罚项能比较有效地解决共线性问题
- 岭回归是一种可用于共线性问题的有偏估计回归方法，实质上是一种改良的最小二乘估计法
- 它通过放弃最小二乘法的无偏性，以损失部分信息、降低精读为代价来获得更实际和可靠性更强的回归系数

In [5]:
model_ridge = Ridge(alpha=1.0)
model_ridge.fit(X, y)
print('coef:\n', model_ridge.coef_)
print('\nintercept:\n', model_ridge.intercept_)

coef:
 [-0.00836161 -0.10320428 -0.01964247 -0.00560743  0.02988531 -0.00410644
  0.00484214 -0.03720753  0.02489295  0.02489295  0.00527641  0.06280543
 -0.00212993 -0.03720753 -0.00726083]

intercept:
 0.6222527142570053


### 2 PCA

- 通过主成分分析，将原始特征转换为少数几个主成分，每个主成分是原特征的线性组合
- 基于主成分做回归分析，可以在不丢失重要数据特征的前提下避开共线性问题

In [6]:
model_pca = PCA()
data_pca = model_pca.fit_transform(X)

# np.cumsum(a, axis=None, dtype=None, out=None)可以在指定的axis上累计求和
ratio_cumsum = np.cumsum(model_pca.explained_variance_ratio_)
print('ratio_cumsum:\n', ratio_cumsum)

# 获取主成分方差占比累积大于0.8的值索引
rule_index = np.where(ratio_cumsum > 0.8)
print('\nrule_index: \n', rule_index)

# rule_index是一个元组，min_index为其最小值
min_index = rule_index[0][0]

# 获取data_pca的所有行，前min_index列.
data_pca_result = data_pca[:, :(min_index + 1)]
print('\nshape:\n', data_pca_result.shape)

ratio_cumsum:
 [0.29512698 0.48842363 0.62481032 0.74903408 0.79338958 0.82974707
 0.865346   0.90040382 0.93489478 0.96783082 1.         1.
 1.         1.         1.        ]

rule_index: 
 (array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14], dtype=int64),)

shape:
 (5000, 6)


In [7]:
model_linear = LinearRegression()
model_linear.fit(data_pca_result, y)

print('coef:', model_linear.coef_)
print('\nintercept', model_linear.intercept_)

coef: [ 0.03679445  0.04293586 -0.08639767 -0.01217279 -0.09520728  0.00861532]

intercept 0.4988
